# Dot Product Program

Group 1 - CEPARCO Project

``Cai, Edison``
``Susada, Stephanie Joy``

Shared Memory in CUDA

### C++ Implementation:

In [ ]:
%%writefile c_dotp.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

//kernel function
void f_dotp(int n, float* h_dotp, float* h_x, float* h_y){
    for(int i = 0; i < n; i++)
        *h_dotp += h_x[i] * h_y[i];
}

int main(){
    const unsigned int ARRAY_SIZE = 1<<24;
    const unsigned ARRAY_BYTES = ARRAY_SIZE * sizeof(float);

    //declare array
    float dotp = 0, *X, *Y;
    X = (float*)malloc(ARRAY_BYTES);
    Y = (float*)malloc(ARRAY_BYTES);

    clock_t start, end;

    //initialize array
    srand(time(0));
    for(int i = 0; i < ARRAY_SIZE; i++){
        X[i] = (rand() % 10) + 1;
        Y[i] = (rand() % 10) + 1;
    }
    
    //print statements for verification
    /*
    printf("X: ");
    for(int i = 0; i < ARRAY_SIZE; i++){
        printf("%f, ", X[i]);
    }
    printf("\nY: ");
    for(int i = 0; i < ARRAY_SIZE; i++){
        printf("%f, ", Y[i]);
    }
    */

    f_dotp(ARRAY_SIZE, &dotp, X, Y);
    dotp = 0;

    start = clock();
    f_dotp(ARRAY_SIZE, &dotp, X, Y);
    end = clock();

    double time_taken = ((double)(end-start))*1e6/ CLOCKS_PER_SEC;

    printf("\ndotp: %f", dotp);

    printf("\nC function took %f us for array size %d \n", time_taken, ARRAY_SIZE);

    //check for errors
    float temp = 0;
    for(int i = 0; i<ARRAY_SIZE; i++){
        temp += X[i] * Y[i];
    }
    if (temp != dotp)
        printf("There is an error with the result.");
    else
        printf("No errors were found.");

    free(X);
    free(Y);
    return 0;

}

Overwriting c_dotp.c


In [ ]:
%%shell
g++ c_dotp.c -o c_dotp

In [ ]:
%%shell
./c_dotp


dotp: 498392928.000000
C function took 51344.000000 us for array size 16777216 
No errors were found.

### CUDA Implementation w/o shared memory using grid-stride loop w/ prefetching + mem advise: 

In [ ]:
%%writefile cuda_dotp_notshared.cu
#include <cuda_profiler_api.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

__global__
void f_dotp(int n, float* h_dotp, float* h_x, float* h_y){
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for(int i = index; i < n; i += stride){
        atomicAdd(h_dotp, (h_x[i]*h_y[i]));
    }
}

int main(){
    const unsigned int ARRAY_SIZE = 1<<20;
    const unsigned ARRAY_BYTES = ARRAY_SIZE * sizeof(float);

    int numThread = 1024;
    int numBlock = (ARRAY_SIZE+numThread-1) / numThread;

    float *dotp = 0, *X, *Y;
    cudaMallocManaged(&X, ARRAY_BYTES);
    cudaMallocManaged(&Y, ARRAY_BYTES);
    cudaMallocManaged(&dotp, sizeof(float));

    srand(time(0));
    for(int i = 0; i < ARRAY_SIZE; i++){
        X[i] = (rand() % 10) + 1;
        Y[i] = (rand() % 10) + 1;
        //X[i] = (float)rand()/RAND_MAX;
        //Y[i] = (float)rand()/RAND_MAX;
    }

    int device = -1;
    cudaGetDevice(&device);
    printf("Device # = %d", device);
    cudaMemPrefetchAsync(X, ARRAY_BYTES, device, NULL);
    cudaMemPrefetchAsync(Y, ARRAY_BYTES, device, NULL);
    cudaMemPrefetchAsync(dotp, ARRAY_BYTES, device, NULL);

    cudaMemAdvise(X, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(Y, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(dotp, sizeof(float), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(X, ARRAY_BYTES, cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(Y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, device);

    /*
    printf("X: ");
    for(int i = 0; i < ARRAY_SIZE; i++){
        printf("%f, ", X[i]);
    }
    printf("\nY: ");
    for(int i = 0; i < ARRAY_SIZE; i++){
        printf("%f, ", Y[i]);
    }
    */

    f_dotp<<<numBlock, numThread>>>(ARRAY_SIZE, dotp, X, Y);
    cudaDeviceSynchronize();

    cudaMemPrefetchAsync(X, ARRAY_BYTES, device, NULL);
    cudaMemPrefetchAsync(Y, ARRAY_BYTES, device, NULL);
    cudaMemPrefetchAsync(dotp, ARRAY_BYTES, device, NULL);

    printf("\ndotp = %f\n", *dotp);

    float temp = 0;
    for(int i = 0; i<ARRAY_SIZE; i++){
        temp += X[i] * Y[i];
    }
    if (temp != *dotp)
        printf("There is an error with the result.\n");
    else
        printf("No errors were found.\n");

    cudaFree(X);
    cudaFree(Y);
    cudaFree(dotp);
    return 0;
}

Overwriting cuda_dotp_notshared.cu


In [ ]:
%%shell
nvcc cuda_dotp_notshared.cu -o cuda_dotp_notshared

In [ ]:
%%shell
nvprof ./cuda_dotp_notshared

==5257== NVPROF is profiling process 5257, command: ./cuda_dotp_notshared
Device # = 0
dotp = 261906.781250
There is an error with the result.
==5257== Profiling application: ./cuda_dotp_notshared
==5257== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  906.09ms         1  906.09ms  906.09ms  906.09ms  f_dotp(int, float*, float*, float*)
      API calls:   85.30%  906.59ms         1  906.59ms  906.59ms  906.59ms  cudaDeviceSynchronize
                   14.51%  154.22ms         3  51.407ms  14.015us  154.17ms  cudaMallocManaged
                    0.07%  732.10us         3  244.03us  64.800us  377.34us  cudaFree
                    0.06%  630.80us         6  105.13us  1.9530us  442.08us  cudaMemPrefetchAsync
                    0.04%  417.95us         5  83.590us  5.0520us  305.49us  cudaMemAdvise
                    0.01%  125.32us       101  1.2400us     133ns  53.897us  cuDeviceGetAttribute
              

### CUDA Implementation w/ Shared Memory using grid-stride loop w/ prefetching + mem advise:

In [ ]:
%%writefile cuda_dotp_shared.cu
#include <cuda_profiler_api.h>
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

__global__
void f_dotp(int n, float* h_dotp, float* h_x, float* h_y){
    __shared__ int t_dotp;
    t_dotp = 0;
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for(int i = index; i < n; i += stride){
        atomicAdd(&t_dotp, (h_x[i]*h_y[i]));
        __syncthreads();
        if(index % blockDim.x == 0)
            atomicAdd(h_dotp, t_dotp);
    }
}

int main(){
    const unsigned int ARRAY_SIZE = 256;
    const unsigned ARRAY_BYTES = ARRAY_SIZE * sizeof(float);

    int numThread = 1024;
    int numBlock = (ARRAY_SIZE+numThread-1) / numThread;

    float *dotp = 0, *X, *Y;
    cudaMallocManaged(&X, ARRAY_BYTES);
    cudaMallocManaged(&Y, ARRAY_BYTES);
    cudaMallocManaged(&dotp, sizeof(float));

    srand(time(0));
    for(int i = 0; i < ARRAY_SIZE; i++){
        X[i] = (rand() % 10) + 1;
        Y[i] = (rand() % 10) + 1;
    }

    int device = -1;
    cudaGetDevice(&device);
    printf("Device # = %d", device);
    cudaMemPrefetchAsync(X, ARRAY_BYTES, device, NULL);
    cudaMemPrefetchAsync(Y, ARRAY_BYTES, device, NULL);
    cudaMemPrefetchAsync(dotp, ARRAY_BYTES, device, NULL);

    cudaMemAdvise(X, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(Y, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(dotp, sizeof(float), cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(X, ARRAY_BYTES, cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(Y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, device);

    /*
    printf("X: ");
    for(int i = 0; i < ARRAY_SIZE; i++){
        printf("%f, ", X[i]);
    }
    printf("\nY: ");
    for(int i = 0; i < ARRAY_SIZE; i++){
        printf("%f, ", Y[i]);
    }
    */

    f_dotp<<<numBlock, numThread>>>(ARRAY_SIZE, dotp, X, Y);
    cudaDeviceSynchronize();

    cudaMemPrefetchAsync(X, ARRAY_BYTES, device, NULL);
    cudaMemPrefetchAsync(Y, ARRAY_BYTES, device, NULL);
    cudaMemPrefetchAsync(dotp, ARRAY_BYTES, device, NULL);

    printf("\ndotp = %f\n", *dotp);

    float temp = 0;
    for(int i = 0; i<ARRAY_SIZE; i++){
        temp += X[i] * Y[i];
    }
    if (temp != *dotp)
        printf("There is an error with the result.\n");
    else
        printf("No errors were found.\n");    

    cudaFree(X);
    cudaFree(Y);
    cudaFree(dotp);
    return 0;
}

Overwriting cuda_dotp_shared.cu


In [ ]:
%%shell
nvcc cuda_dotp_shared.cu -o cuda_dotp_shared

In [ ]:
%%shell
nvprof ./cuda_dotp_shared

==2463== NVPROF is profiling process 2463, command: ./cuda_dotp_shared
Device # = 0

dotp = 7743.000000
No errors were found.
==2463== Profiling application: ./cuda_dotp_shared
==2463== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  79.902us         1  79.902us  79.902us  79.902us  f_dotp(int, float*, float*, float*)
      API calls:   99.67%  245.60ms         3  81.866ms  4.8230us  245.57ms  cudaMallocManaged
                    0.14%  344.53us         6  57.422us  1.3670us  210.77us  cudaMemPrefetchAsync
                    0.06%  148.72us         3  49.572us  6.7950us  111.02us  cudaFree
                    0.05%  116.22us       101  1.1500us     128ns  48.812us  cuDeviceGetAttribute
                    0.03%  83.754us         1  83.754us  83.754us  83.754us  cudaDeviceSynchronize
                    0.01%  33.855us         1  33.855us  33.855us  33.855us  cudaLaunchKernel
                    0.01%  30.5